# About
In this notebook we'll be building an Alpaca-based dataset containing every comment scraped from r/Hasan_Piker, specifically prepared for finetuning Llama3.1.

WIP

## 1.0 - Preparation

### 1.1 - Import Packages

In [94]:
# import libs
import json
import pandas as pd
from loguru import logger

### 1.2 - Set Configuration Variables
Change these to build different datasets

In [ ]:
SUBREDDIT = "Destiny"
CLEANED_DATA = '../../json/output/cleaned_data_DEVELOPMENT.json'

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

## 2.0 - Filter & Transform

### 2.1 - Load Cleaned Data

In [ ]:
# load data into pandas dataframe
with open(CLEANED_DATA, 'r') as f:
  data = json.load(f)

df = pd.DataFrame(data)

# logging
prefilter_len = len(df)
logger.info(f"Data loaded: {df.shape}")
df.head(1)

2024-11-24 13:05:33.874 | INFO     | __main__:<module>:8 - Data loaded: (46, 25)


,id,subreddit,selftext,title,downs,name,upvote_ratio,ups,removed_by_category,link_flair_text,...,no_follow,created_utc,author_flair_text,author,num_comments,subreddit_subscribers,send_replies,is_video,deleted,comments
0,1dx1b0z,Destiny,,New Vegan,0,t3_1dx1b0z,0.95,121,None,Shitpost,...,False,1720304607,None,TuningsGaming,2,248289,True,False,False,"[{'id': 'lbyv8mn', 'total_awards_received': 0,..."


### 2.2 - Drop Unwanted Subreddits

In [ ]:
# only keep the rows where the subreddit is in SUBREDDITS
df = df[df['subreddit'] == SUBREDDIT]

# logging
posts_removed = prefilter_len - len(df)
logger.info(f"Data filtered: {df.shape}")
logger.info(f"{posts_removed} post{'s' if posts_removed > 1 else ''} removed")
df.head(1)

2024-11-24 13:05:33.890 | INFO     | __main__:<module>:6 - Data filtered: (45, 25)
2024-11-24 13:05:33.891 | INFO     | __main__:<module>:7 - 1 post removed


,id,subreddit,selftext,title,downs,name,upvote_ratio,ups,removed_by_category,link_flair_text,...,no_follow,created_utc,author_flair_text,author,num_comments,subreddit_subscribers,send_replies,is_video,deleted,comments
0,1dx1b0z,Destiny,,New Vegan,0,t3_1dx1b0z,0.95,121,None,Shitpost,...,False,1720304607,None,TuningsGaming,2,248289,True,False,False,"[{'id': 'lbyv8mn', 'total_awards_received': 0,..."


## 3.0 - Bucket Data by Instruction Type

### 3.1 - Define instructions

In [ ]:
instructions = {
  f"Write a response to this comment from r/{SUBREDDIT}": []
}

# TODO: MAKE INSTRUCTIONS MORE SPECIFIC LIKE:
# instructions = [
#   { 
#     "instruction": f"Write a controverial response to this comment from r/{SUBREDDIT}",
#     "response": []
#   },
#   { 
#     "instruction": f"Write a likely upvoted response to this comment from r/{SUBREDDIT}",
#     "responses": []
#   },
#   { 
#     "instruction": f"Write a likely downvoted response to this comment from r/{SUBREDDIT}",
#     "responses": []
#   },
#   { 
#     "instruction": f"Write a response that is likely to be removed by the moderators to this comment from r/{SUBREDDIT}",
#     "responses": []
#   }
# ]